In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import censusdata
import re

censuskey = '7b3e8a6afe58867abcf5aecad26845d802115f1f'

In [3]:
#Define Funtions

#Funcion For NonDigit Removing Characters from Row - Used for Extracting GEOID
def remove_chars(s):
    clean = int(re.sub('[^0-9]+', '', str(s))[-11:])
    return clean

#Function to get census data according to requirements
def getcensustables(table):
    tablename = 'data_' + table
    print(f'Downloading {table}')
    
    # Set requirements to pull from Census API
    
    variablestopull = allvariablescsv.loc[allvariablescsv['Group']==table]['Name'].tolist()
    listofcounties = ['121', '089']
    yearstopull = [2017] #range(2018,2018,1)

    #Pull Census Data according to requirement above
    atlantadata = pd.DataFrame()

    for year in yearstopull:
        for county in listofcounties:

            newdata = censusdata.download('acs5', year, censusdata.censusgeo([('state', '13'), ('county', county), ('tract', '*')]),variablestopull,censuskey)
            type(newdata)
            newdata['YEAR'] = year
            atlantadata = atlantadata.append(newdata)

    #Rename Column Headings from Code to Text
    for variable in variablestopull:
        #print(variable)
        renamevar1 = allvariablescsv.loc[allvariablescsv['Name']==variable]['Label'].iloc[0]
        renamevar2 = allvariablescsv.loc[allvariablescsv['Name']==variable]['Concept'].iloc[0]
        renamevar = renamevar2 + '_'+ renamevar1
        #print(renamevar)
        atlantadata = atlantadata.rename(columns={variable: renamevar})

    #Transform Index to create GeoID

    #Create GeoID Column
    atlantadata['Name2'] = atlantadata.index
    atlantadata['GEOID'] = atlantadata['Name2'].apply(remove_chars)
    atlantadata.drop(columns=['Name2'], inplace=True)
    
    #Only Keep Atlanta Census Tracts(Geocodes)
    finaldata = atlantageocodes.merge(atlantadata, how='left', on='GEOID')
    
    finaldata = finaldata.dropna(axis=0, how='any')
    
    return finaldata

In [4]:
#Code Begins here

#Tables to Load from data/tablestopull.csv
tablestoloadcsv = pd.read_csv('data/tablestopull.csv')
tablestoload = tablestoloadcsv['Tables'].tolist()

#Uncommnent Line below to override csv
#tablestoload = ['B19013','B01003','B11001', 'B25064', 'B25074','B25063', 'B25061','B25056','B00002','B25001','B25122']

#Allvariables Table to get table detail
allvariablescsv = pd.read_csv('data/allvariables.csv')

#Atlanta Geo Codes Table to filter to city of atlanta only 
atlantageocodes = pd.read_csv('data/atlantageocodes.csv')

#Create dictionary with each cleaned table as an entry
alltables = dict()  


#Script will attempt to load tables from data folder first, otherwise download them
for table in tablestoload:
    try:
        data = pd.read_csv('data/' + table + '.csv')
        alltables[table] = data
        print(f'Loading {table}')
    except:
        data = getcensustables(table)
        alltables[table] = data

        data.to_csv('data/' + table + '.csv')
        
        

Loading B19013
Loading B01003
Loading B11001
Loading B25064
Loading B25074
Loading B25063
Loading B25061
Loading B25056
Loading B00002
Loading B25001
Loading B25122
Loading B19119
Loading B19019
Loading B25106


In [15]:
#Pull HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS and group by year
percentrentbyincome = alltables['B25074']


display(percentrentbyincome) ### for class


percentrentbyincome = percentrentbyincome.groupby('YEAR').sum()
percentrentbyincome = percentrentbyincome.transpose()


#Split index into variables
percentrentbyincome['YEAR'] = percentrentbyincome.index
split1 = percentrentbyincome['YEAR'].str.split('PAST 12 MONTHS_Estimate!!Total!!', expand=True)
split2 = split1[1].str.split('!!', expand=True)

percentrentbyincome['Income'] = split2[0]
percentrentbyincome['Rent as % of Income'] = split2[1]

percentrentbyincome.drop(columns='YEAR')


#Set years to loop through
years = [2017.0]

percentrentbyincome2 = percentrentbyincome.copy()


# for year in years:

#     print(year)
#     percentrentbyincome = percentrentbyincome2[[year,'Income','Rent as % of Income']]

#     #display(percentrentbyincome)
#     percentrentbyincome = percentrentbyincome.dropna(axis=0, how='any')
#     #display(percentrentbyincome)
#     # display(percentrentbyincome.pivot(index=percentrentbyincome['Income'], columns=percentrentbyincome['Rent Change'], values=percentrentbyincome[2012.0]))

#     percentrentbyincomepv = pd.crosstab(index=percentrentbyincome.iloc[:,1], columns=percentrentbyincome.iloc[:,2], 
#                       values=percentrentbyincome.iloc[:,0], aggfunc='mean')

#     percentrentbyincomepv['TotalHHs'] = percentrentbyincomepv.sum(axis = 1, skipna = True) 

#     #percentrentbyincomepv = percentrentbyincomepv.div(percentrentbyincomepv.TotalHHs, axis=0)*100

#     display(percentrentbyincomepv)

,Unnamed: 0,GEOID,"HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!Less than 20.0 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!20.0 to 24.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!25.0 to 29.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!30.0 to 34.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!35.0 to 39.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!40.0 to 49.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!50.0 percent or more",...,"HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$75,000 to $99,999!!50.0 percent or more","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!Less than 20.0 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!20.0 to 24.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!25.0 to 29.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!30.0 to 34.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!35.0 to 39.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!40.0 to 49.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!50.0 percent or more",YEAR
0,3,13121006601,108,0,0,7,0,0,0,66,...,0,3,3,0,0,0,0,0,0,2017
1,7,13121006700,217,40,20,17,0,0,0,57,...,0,12,12,0,0,0,0,0,0,2017
2,11,13121009300,190,0,0,0,0,0,0,116,...,0,312,278,34,0,0,0,0,0,2017
3,15,13121009402,194,0,0,0,0,0,0,112,...,0,632,567,65,0,0,0,0,0,2017
4,20,13089023700,323,0,0,20,0,0,0,236,...,0,141,110,31,0,0,0,0,0,2017
5,24,13121001300,123,0,0,0,0,0,0,123,...,0,280,251,29,0,0,0,0,0,2017
6,28,13121001400,60,0,0,0,0,0,0,16,...,0,186,174,12,0,0,0,0,0,2017
7,32,13121001800,627,0,0,111,86,0,121,161,...,0,201,178,23,0,0,0,0,0,2017
8,36,13121001600,64,0,0,0,0,0,0,16,...,0,261,222,39,0,0,0,0,0,2017
9,40,13121001500,279,0,0,19,16,0,0,168,...,0,205,177,28,0,0,0,0,0,2017


2017.0


Rent as % of Income,20.0 to 24.9 percent,25.0 to 29.9 percent,30.0 to 34.9 percent,35.0 to 39.9 percent,40.0 to 49.9 percent,50.0 percent or more,Less than 20.0 percent,TotalHHs
Income,,,,,,,,
"$10,000 to $19,999",882.0,1241.0,806.0,468.0,1541.0,10248.0,921.0,16107.0
"$100,000 or more",1374.0,491.0,80.0,96.0,65.0,0.0,11984.0,14090.0
"$20,000 to $34,999",665.0,1637.0,2827.0,2743.0,4552.0,5058.0,972.0,18454.0
"$35,000 to $49,999",2332.0,4086.0,3172.0,1859.0,1560.0,809.0,1091.0,14909.0
"$50,000 to $74,999",5172.0,3830.0,1917.0,1159.0,624.0,169.0,4113.0,16984.0
"$75,000 to $99,999",2841.0,1116.0,350.0,200.0,52.0,12.0,4552.0,9123.0
"Less than $10,000",421.0,897.0,536.0,219.0,600.0,9194.0,400.0,12267.0


In [30]:
#Pull HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS and group by year
percentrentbyincome = alltables['B19119']


display(percentrentbyincome) ### for class


percentrentbyincome = percentrentbyincome.groupby('YEAR').sum()
percentrentbyincome = percentrentbyincome.transpose()


#Split index into variables
percentrentbyincome['YEAR'] = percentrentbyincome.index
split1 = percentrentbyincome['YEAR'].str.split('PAST 12 MONTHS_Estimate!!Total!!', expand=True)
split2 = split1[1].str.split('!!', expand=True)

percentrentbyincome['Income'] = split2[0]
percentrentbyincome['Rent as % of Income'] = split2[1]

percentrentbyincome.drop(columns='YEAR')


#Set years to loop through
years = [2017.0]

percentrentbyincome2 = percentrentbyincome.copy()


# for year in years:

#     print(year)
#     percentrentbyincome = percentrentbyincome2[[year,'Income','Rent as % of Income']]

#     #display(percentrentbyincome)
#     percentrentbyincome = percentrentbyincome.dropna(axis=0, how='any')
#     #display(percentrentbyincome)
#     # display(percentrentbyincome.pivot(index=percentrentbyincome['Income'], columns=percentrentbyincome['Rent Change'], values=percentrentbyincome[2012.0]))

#     percentrentbyincomepv = pd.crosstab(index=percentrentbyincome.iloc[:,1], columns=percentrentbyincome.iloc[:,2], 
#                       values=percentrentbyincome.iloc[:,0], aggfunc='mean')

#     percentrentbyincomepv['TotalHHs'] = percentrentbyincomepv.sum(axis = 1, skipna = True) 

#     #percentrentbyincomepv = percentrentbyincomepv.div(percentrentbyincomepv.TotalHHs, axis=0)*100

#     display(percentrentbyincomepv)

,GEOID,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE_Estimate!!Total,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE_Estimate!!Total!!2-person families,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE_Estimate!!Total!!3-person families,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE_Estimate!!Total!!4-person families,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE_Estimate!!Total!!5-person families,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE_Estimate!!Total!!6-person families,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE_Estimate!!Total!!7-or-more-person families,YEAR
0,13121006601,51375.0,60069.0,49000.0,-666666666.0,52188.0,-666666666.0,-666666666.0,2017.0
1,13121006700,37821.0,20347.0,39844.0,41212.0,-666666666.0,-666666666.0,-666666666.0,2017.0
2,13121009300,171452.0,167760.0,199028.0,207237.0,-666666666.0,-666666666.0,-666666666.0,2017.0
3,13121009402,96484.0,105500.0,72955.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,2017.0
5,13089023700,45919.0,47297.0,42831.0,51250.0,-666666666.0,-666666666.0,-666666666.0,2017.0
6,13121001300,100104.0,77841.0,113611.0,177750.0,-666666666.0,-666666666.0,-666666666.0,2017.0
7,13121001400,128438.0,126406.0,131250.0,155125.0,-666666666.0,-666666666.0,-666666666.0,2017.0
8,13121001800,34375.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,2017.0
9,13121001600,111290.0,86442.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,2017.0
10,13121001500,131417.0,-666666666.0,185714.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,2017.0


KeyError: 1

In [34]:
#Pull HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS and group by year
percentrentbyincome = alltables['B19019']


display(percentrentbyincome) ### for class


percentrentbyincome = percentrentbyincome.groupby('YEAR').sum()
percentrentbyincome = percentrentbyincome.transpose()


#Split index into variables
percentrentbyincome['YEAR'] = percentrentbyincome.index
split1 = percentrentbyincome['YEAR'].str.split('PAST 12 MONTHS_Estimate!!Total!!', expand=True)
split2 = split1[1].str.split('!!', expand=True)

percentrentbyincome['Income'] = split2[0]
percentrentbyincome['Rent as % of Income'] = split2[1]

percentrentbyincome.drop(columns='YEAR')


#Set years to loop through
years = [2017.0]

percentrentbyincome2 = percentrentbyincome.copy()


# for year in years:

#     print(year)
#     percentrentbyincome = percentrentbyincome2[[year,'Income','Rent as % of Income']]

#     #display(percentrentbyincome)
#     percentrentbyincome = percentrentbyincome.dropna(axis=0, how='any')
#     #display(percentrentbyincome)
#     # display(percentrentbyincome.pivot(index=percentrentbyincome['Income'], columns=percentrentbyincome['Rent Change'], values=percentrentbyincome[2012.0]))

#     percentrentbyincomepv = pd.crosstab(index=percentrentbyincome.iloc[:,1], columns=percentrentbyincome.iloc[:,2], 
#                       values=percentrentbyincome.iloc[:,0], aggfunc='mean')

#     percentrentbyincomepv['TotalHHs'] = percentrentbyincomepv.sum(axis = 1, skipna = True) 

#     #percentrentbyincomepv = percentrentbyincomepv.div(percentrentbyincomepv.TotalHHs, axis=0)*100

#     display(percentrentbyincomepv)

,GEOID,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY HOUSEHOLD SIZE_Estimate!!Total,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY HOUSEHOLD SIZE_Estimate!!Total!!1-person households,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY HOUSEHOLD SIZE_Estimate!!Total!!2-person households,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY HOUSEHOLD SIZE_Estimate!!Total!!3-person households,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY HOUSEHOLD SIZE_Estimate!!Total!!4-person households,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY HOUSEHOLD SIZE_Estimate!!Total!!5-person households,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY HOUSEHOLD SIZE_Estimate!!Total!!6-person households,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY HOUSEHOLD SIZE_Estimate!!Total!!7-or-more-person households,YEAR
0,13121006601,36250.0,17500.0,46923.0,70313.0,-666666666.0,52188.0,-666666666.0,-666666666.0,2017.0
1,13121006700,23017.0,20250.0,20389.0,39188.0,41742.0,-666666666.0,-666666666.0,-666666666.0,2017.0
2,13121009300,100844.0,55637.0,148162.0,240341.0,206645.0,-666666666.0,-666666666.0,-666666666.0,2017.0
3,13121009402,69034.0,54530.0,106721.0,118750.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,2017.0
5,13089023700,45077.0,19575.0,56701.0,45671.0,51917.0,-666666666.0,-666666666.0,-666666666.0,2017.0
6,13121001300,66223.0,46806.0,91750.0,-666666666.0,177750.0,-666666666.0,-666666666.0,-666666666.0,2017.0
7,13121001400,86333.0,62357.0,140446.0,131250.0,155125.0,-666666666.0,-666666666.0,-666666666.0,2017.0
8,13121001800,42159.0,39545.0,60288.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,2017.0
9,13121001600,66806.0,48750.0,83594.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,2017.0
10,13121001500,63464.0,40848.0,100417.0,210865.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,2017.0


KeyError: 1

In [22]:
#Pull HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS and group by year
percentrentbyincome = alltables['B25106']

percentrentbyincome = percentrentbyincome[percentrentbyincome.columns.drop(list(percentrentbyincome.filter(regex='Owner')))]
percentrentbyincome = percentrentbyincome[percentrentbyincome.columns.drop(list(percentrentbyincome.filter(regex='percent')))]

display(percentrentbyincome) ### for class

lowincomehhs = pd.DataFrame()

# display(percentrentbyincome.columns)

# lowincomehouseholds['TotalHHs'] = percentrentbyincome

# only keep the related columns
cols_of_sum=['TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!Less than $20,000',\
                  'TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!$20,000 to $34,999',\
                  'TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!$35,000 to $49,999']

# lowincomehhs = percentrentbyincome[cols_of_interest]
lowincomehhs['GEOID'] = percentrentbyincome['GEOID']
lowincomehhs['TotalLowIncomeHHs'] = percentrentbyincome[cols_of_sum].sum(axis=1)

# clean_HUDcsv.rename(columns={'Code': 'GEOID'},inplace=True)
lowincomehhs

,Unnamed: 0,GEOID,TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total,TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units,"TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!Less than $20,000","TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!$20,000 to $34,999","TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!$35,000 to $49,999","TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!$50,000 to $74,999","TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!$75,000 or more",TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!Zero or negative income,TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!No cash rent,YEAR
0,0,13121006601,764.0,332.0,154.0,36.0,53.0,37.0,9.0,14.0,29.0,2017.0
1,1,13121006700,1505.0,954.0,417.0,194.0,88.0,136.0,36.0,58.0,25.0,2017.0
2,2,13121009300,2439.0,1044.0,139.0,175.0,28.0,188.0,425.0,74.0,15.0,2017.0
3,3,13121009402,2487.0,2130.0,232.0,261.0,182.0,498.0,836.0,82.0,39.0,2017.0
4,5,13089023700,2363.0,1398.0,478.0,169.0,266.0,178.0,240.0,67.0,0.0,2017.0
5,6,13121001300,2235.0,1391.0,254.0,164.0,262.0,265.0,446.0,0.0,0.0,2017.0
6,7,13121001400,1300.0,711.0,54.0,75.0,56.0,183.0,299.0,44.0,0.0,2017.0
7,8,13121001800,3021.0,2138.0,770.0,270.0,236.0,293.0,391.0,148.0,30.0,2017.0
8,9,13121001600,1304.0,1002.0,72.0,169.0,152.0,171.0,390.0,48.0,0.0,2017.0
9,10,13121001500,2437.0,1547.0,317.0,351.0,182.0,209.0,401.0,56.0,31.0,2017.0


,GEOID,TotalLowIncomeHHs
0,13121006601,243.0
1,13121006700,699.0
2,13121009300,342.0
3,13121009402,675.0
4,13089023700,913.0
5,13121001300,680.0
6,13121001400,185.0
7,13121001800,1276.0
8,13121001600,393.0
9,13121001500,850.0
